In [2]:
#pytania: czy odległość ma być liczbą całkowitą, czy mamy zaczynać od któregoś konkretnego wierzchołka 

In [1]:
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

# Load instance

In [2]:
instance_number = 0  # 0 - "kroA100.tsp"    1 - "kroB100.tsp"
instances_names = ["kroA100.tsp","kroB100.tsp"]
data = np.genfromtxt(f'instances/{instances_names[instance_number]}', skip_header=6, skip_footer=1, dtype='int64')[:,1:]

# Calculating distance matrix

In [3]:
def calc_distance_matrix(data):
    dist_matrix = distance_matrix(data, data)
    dist_matrix = np.around(dist_matrix, decimals=0)
    dist_matrix = dist_matrix.astype(int)
    return dist_matrix

In [4]:
distance_matrix = calc_distance_matrix(data)
distance_matrix

array([[   0, 1693, 2252, ...,  783, 1468, 2643],
       [1693,    0, 1708, ..., 1911, 3156, 1831],
       [2252, 1708,    0, ..., 1786, 3315,  454],
       ...,
       [ 783, 1911, 1786, ...,    0, 1535, 2228],
       [1468, 3156, 3315, ..., 1535,    0, 3760],
       [2643, 1831,  454, ..., 2228, 3760,    0]])

# GREEDY

In [84]:
def greedy_algorithm(N, start_point):
    
    n = int(np.ceil(N / 2))
    visited = np.array([start_point])
    total_distance = 0
    point = start_point
    
    for i in range(n-1):
        sorted_distances = np.argsort(distance_matrix[point])[1:]
        new_point = sorted_distances[~np.in1d(sorted_distances, visited)][0]

        visited = np.append(visited, [new_point])
        distance = distance_matrix[point, new_point]
        total_distance += distance
        point = new_point
        
        
    visited = np.append(visited, [start_point])
    total_distance += distance_matrix[point, start_point]
    
    return (total_distance, visited)

In [85]:
N = 100
start_point = 0

total_distance, visited = greedy_algorithm(N, start_point)

In [86]:
print(total_distance)
print(visited)

14094
[ 0 62  5 48 89  9 83 71 20 73 58 16 14 10 31 90 97 22 44 46 92 27 66 57
 60 24 80 68 72 49 43  1 53 39 63 67 84 38 29 95 77 51  4 36 32 75 12 94
 81 47  0]
